In [3]:
import requests
import pandas as pd

In [1]:
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir("/content/drive/MyDrive/KT aivleschool/KT_aivle_school_AI_track/14. MINI_PJ4")
!ls

 1on1question_label				      kobert.ipynb
'1. 문의 내용 유형 분류_분석_전처리_에이블러.ipynb'   ko.zip
'2. 문의 내용 유형 분류_모델링_에이블러.ipynb'	      random_submission.csv
 BERT.ipynb					      stopwords-ko.txt
 BERT_model_hayden.pt				      test.csv
 bert_model.pt					      train.csv
 BERT_제출.ipynb				      전처리_강사님_코드.ipynb
'kobert copy.ipynb'


In [ ]:
train = pd.read_csv('train.tsv', delimiter="\t", index_col = 0)
val = pd.read_csv('val.tsv', delimiter="\t", index_col = 0)
test = pd.read_csv('test.tsv', delimiter="\t", index_col = 0)

In [ ]:
train

,0,1
0,실력 부족 실시간 수업 번 이해 복습 영상 보 복습 영상 올라오 당일 아닌 익일 시...,3
1,안녕 가우스 정다각형 문제 질문 드립니다 정수 k 받 페르마 소수 제곱수 리스트 받...,0
2,안녕 코딩 마스터스 채점 기 눌렸 passed 떠서 제출 했 틀렸 나왔 이러 경우 ...,0
3,aidu 저장 공지 받 튜 터 aivle edu 제출 해야,3
4,cmd 창 장고 실행 됩니다 python manage py runserver 오류 ...,1
...,...,...
2774,질문 드려서 죄송 view for 문 돌 results result 담 result...,1
2775,모델 가중치 로딩 모델 저장 모델 파라미터 epochs 필수 번 정의 모델 저장,0
2776,vscode 터미널 python manage py makemigrations pyt...,1
2777,만들 인 스턴스 종료 되 연결 버튼 활성 되 해야,1


In [ ]:
train = pd.concat([train, val])

In [ ]:
train_x = train.iloc[:,0].tolist()
test_x = test.iloc[:,0].tolist() 

In [ ]:
train.shape

(3706, 2)

In [ ]:
import numpy as np

train_x_split = [s.split() for s in train_x]

test_x_split = [s.split() for s in test_x]

In [ ]:
type(train_x_split)

list

In [ ]:
train_y = train.iloc[:,1].values

In [ ]:
type(train_y)

numpy.ndarray

In [ ]:
!conda install -y gensim

/bin/bash: conda: command not found


In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
model = Word2Vec(train_x_split,         # 리스트 형태의 데이터
                 sg=1,         # 0: CBOW, 1: Skip-gram
                 vector_size=64,     # 벡터 크기#100
                 window=3,     # 고려할 앞뒤 폭(앞뒤 3단어)
                 min_count=3,  # 사용할 단어의 최소 빈도(3회 이하 단어 무시)
                 workers=4)    # 동시에 처리할 작업 수(코어 수와 비슷하게 설정)

In [ ]:
model.save('word2vec.model')

In [ ]:
model = Word2Vec.load('word2vec.model')

In [ ]:
model.wv['에이블']

array([ 0.16756962, -0.42359516,  0.2025853 ,  0.0352915 , -0.31209767,
        0.30220214, -0.18713656,  0.19234224, -0.11771297, -0.0194612 ,
       -0.25601777,  0.13863598, -0.44787437, -0.00501329,  0.4153747 ,
        0.12665837,  0.00624849, -0.00135183,  0.15295379,  0.09767311,
        0.3711557 ,  0.00047624, -0.0692672 , -0.38305756,  0.15277404,
        0.1970392 ,  0.09459851,  0.08717863,  0.28493547,  0.2859381 ,
        0.12103452,  0.41396806,  0.01682273, -0.07876593, -0.11324058,
       -0.11083763, -0.07087989, -0.14465122,  0.13502942,  0.04085305,
        0.21701166, -0.04081702, -0.04055858, -0.05832429, -0.18182993,
       -0.14626585,  0.01764977,  0.05725374, -0.03276821,  0.31056735,
       -0.01174361, -0.15447736,  0.22097251,  0.19279055, -0.18894386,
       -0.00749823,  0.06563925,  0.15901393, -0.20918767,  0.32982805,
       -0.3694201 ,  0.01070786, -0.04164318,  0.42650047], dtype=float32)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding

In [ ]:
NUM_WORDS, EMB_DIM = model.wv.vectors.shape
MAX_LEN = 120 #50
PADDING_TYPE = "pre"

In [ ]:
emb = Embedding(input_dim = NUM_WORDS, # 사용할 토큰 개수 
                output_dim=EMB_DIM, # 한 token당 몇차원(몇개)의 특징값으로 바꾸어낼 것인가.
                trainable=False, weights=[model.wv.vectors])

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
t = Tokenizer(num_words = NUM_WORDS)
t.fit_on_texts(train_x)
encoded_train_x = t.texts_to_sequences(train_x)
encoded_test_x = t.texts_to_sequences(test_x)
print(encoded_train_x)
print(encoded_test_x)

[[2842, 656, 1748, 199, 16, 50, 673, 335, 84, 673, 335, 2278, 1749, 310, 3337, 169, 311, 50, 220, 910, 199, 587, 318, 910, 199, 50, 418, 2, 1749, 169, 169, 265, 710, 31, 674, 123, 516, 1, 2542, 2543, 178], [24, 11, 30, 10, 588, 104, 54, 1522, 182, 54, 182, 4, 105, 95, 265, 39, 54, 104, 182, 4, 1, 59, 7, 752, 318, 214, 56, 174, 47, 504, 90, 675, 25, 26, 26, 12, 104, 45, 40, 2843, 2843, 4, 2279, 4, 21, 5, 6, 48, 2279, 5, 21, 5, 6, 2279, 21, 85, 6, 2843, 4, 21, 179, 6, 2843, 17, 85, 179, 4, 85, 179, 5, 4, 61, 319, 4, 17, 104, 6, 4, 13, 752, 126, 13, 214, 26, 397, 59, 174, 1188, 63, 174, 1188, 63, 174, 387, 63, 174, 1188, 63, 174, 1188, 63, 174, 1188, 63, 174, 387, 63], [24, 147, 432, 397, 23, 1188, 380, 52, 87, 773, 303, 492, 34, 34, 124, 1, 1335, 12, 773, 57, 25], [850, 134, 2280, 54, 221, 196, 517, 2077, 52, 74], [657, 229, 1750, 44, 152, 42, 438, 46, 617, 27, 1626, 37, 589, 26, 1413, 517, 1252, 1336, 37, 589, 517, 1252, 1336, 26, 42, 438, 46, 617, 2281, 21, 66, 1894, 245, 2544, 2844, 1

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_train_x = pad_sequences(encoded_train_x, maxlen = MAX_LEN, padding = PADDING_TYPE) # padding을 없앨 수도
padded_test_x = pad_sequences(encoded_test_x, maxlen = MAX_LEN, padding = PADDING_TYPE)
padded_train_x[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 2842,  656, 1748,  199,   16,   50,  673,  335,   84,
        673,  335, 2278, 1749,  310, 3337,  169,  311,   50,  220,  910,
        199,  587,  318,  910,  199,   50,  418,    2, 1749,  169,  169,
        265,  710,   31,  674,  123,  516,    1, 2542, 2543,  178],
      dtype=int32)

In [ ]:
not0train_idx = padded_train_x.sum(1) > 0
not0test_idx = padded_test_x.sum(1) > 0

padded_train_x = padded_train_x[not0train_idx]
train_y = train_y[not0train_idx]

padded_test_x = padded_test_x[not0test_idx]

In [ ]:
from tensorflow.keras.utils import to_categorical
one_hot_train_y = to_categorical(train_y, 5)

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool2D, Bidirectional, LSTM, SimpleRNN, GRU

from tensorflow.keras.models import Sequential

In [ ]:
from keras.layers import Dropout

# emb = Embedding(input_dim = NUM_WORDS, # 사용할 토큰 개수 
#                 output_dim=EMB_DIM, # 한 token당 몇차원(몇개)의 특징값으로 바꾸어낼 것인가.
#                 trainable=False, weights=[model.wv.vectors])

model = tf.keras.Sequential([
    Embedding(input_dim = NUM_WORDS, # 사용할 토큰 개수 
                output_dim=EMB_DIM),
    Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
    Bidirectional(tf.keras.layers.LSTM(32)),
    Dense(64, activation = "relu"),
    Dropout(0.5),
    Dense(5, activation = "softmax")
])

history = model.compile(loss = 'categorical_crossentropy',
              optimizer = keras.optimizers.Adam(0.05),
              metrics = 'accuracy')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          253312    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 5)                 3

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=5,
                   verbose=1)

model.fit(padded_train_x, one_hot_train_y, epochs=1000,
          verbose =1,
          callbacks=[es],
          validation_split=0.2)

Epoch 1/1000
93/93 [==============================] - 20s 132ms/step - loss: 1.3732 - accuracy: 0.4339 - val_loss: 1.0147 - val_accuracy: 0.5816
Epoch 2/1000
93/93 [==============================] - 11s 116ms/step - loss: 0.8060 - accuracy: 0.7105 - val_loss: 0.6865 - val_accuracy: 0.7463
Epoch 3/1000
93/93 [==============================] - 11s 119ms/step - loss: 0.4794 - accuracy: 0.8573 - val_loss: 0.6071 - val_accuracy: 0.8084
Epoch 4/1000
93/93 [==============================] - 11s 120ms/step - loss: 0.2812 - accuracy: 0.9295 - val_loss: 0.7240 - val_accuracy: 0.8016
Epoch 5/1000
93/93 [==============================] - 10s 110ms/step - loss: 0.1847 - accuracy: 0.9555 - val_loss: 0.8159 - val_accuracy: 0.7773
Epoch 6/1000
93/93 [==============================] - 10s 111ms/step - loss: 0.1519 - accuracy: 0.9642 - val_loss: 0.8094 - val_accuracy: 0.8043
Epoch 7/1000
93/93 [==============================] - 10s 112ms/step - loss: 0.1022 - accuracy: 0.9760 - val_loss: 0.9178 - val_ac

In [ ]:
y_pred = model.predict(padded_test_x)

30/30 [==============================] - 3s 40ms/step


In [ ]:
np.argmax(y_pred, axis=1)

array([3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 2, 3, 0, 1, 1, 0, 0, 3,
       0, 3, 0, 0, 3, 0, 0, 0, 1, 0, 4, 4, 0, 2, 2, 0, 2, 0, 0, 2, 0, 3,
       3, 2, 0, 3, 1, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 2, 0,
       0, 3, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 2, 0, 0, 3, 0, 1, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 1, 1, 0, 0, 0, 3, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 2, 1, 2, 0, 2, 0, 0, 3, 0, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 3, 0, 3, 0, 2, 0, 2, 3, 1, 2, 2, 2, 0, 3, 2, 1, 0, 0,
       2, 0, 2, 1, 0, 0, 2, 0, 0, 0, 4, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 3, 3, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0,
       4, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 4, 3, 2, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 3, 2, 0, 3, 2, 0, 3, 0, 3, 0, 0,

In [ ]:
result = pd.DataFrame({'id':[i for i in range(0, len(y_pred))], 'label': np.argmax(y_pred, axis=1)})

In [ ]:
result.to_csv("result.csv", index = False)

In [ ]:
pd.read_csv("result.csv")

,id,laebl
0,0,3
1,1,3
2,2,0
3,3,0
4,4,0
...,...,...
924,924,3
925,925,0
926,926,3
927,927,1
